In [1]:
#Imports
import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import MinMaxScaler

In [2]:
#Load dataset and define maximum columns to be displayed so as to avoid some columns from being hidden
data = pd.read_json("S:/Xampp/htdocs/tweepy/tweets2.json")
pd.set_option('display.max_columns', 500)

In [3]:
#Shuffle the data, to bring records with coordinates set to the top 10
data = data.sample(frac=1).reset_index(drop=True)
data.head(10)

,contributors,coordinates,created_at,display_text_range,entities,extended_entities,extended_tweet,favorite_count,favorited,filter_level,geo,id,id_str,in_reply_to_screen_name,in_reply_to_status_id,in_reply_to_status_id_str,in_reply_to_user_id,in_reply_to_user_id_str,is_quote_status,lang,limit,place,possibly_sensitive,quote_count,quoted_status,quoted_status_id,quoted_status_id_str,quoted_status_permalink,reply_count,retweet_count,retweeted,source,text,timestamp_ms,truncated,user
0,NaN,None,2019-03-10 16:46:16,NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,"{'full_text': 'U don't understand my feelings,...",0.0,0.0,low,None,1.104786e+18,1.104786e+18,None,NaN,NaN,NaN,NaN,0.0,en,NaN,"{'id': '7519d5ef0b317d8f', 'url': 'https://api...",NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...","U don't understand my feelings, emotions and m...",2019-03-10 16:46:16.741,1.0,"{'id': 3772522093, 'id_str': '3772522093', 'na..."
1,NaN,None,2019-03-10 16:45:46,"[0, 140]","{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,{'full_text': 'Traveling back to Saint Francis...,0.0,0.0,low,None,1.104785e+18,1.104785e+18,None,NaN,NaN,NaN,NaN,0.0,en,NaN,"{'id': '512a8a4a4c4b4be0', 'url': 'https://api...",0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Traveling back to Saint Francis U for the star...,2019-03-10 16:45:46.659,1.0,"{'id': 736617558921646080, 'id_str': '73661755..."
2,NaN,None,2019-03-10 16:46:33,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,0.0,low,None,1.104786e+18,1.104786e+18,None,NaN,NaN,NaN,NaN,0.0,en,NaN,"{'id': '017a4afa29d71c65', 'url': 'https://api...",NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",don't need your company. I can do things by my...,2019-03-10 16:46:33.830,0.0,"{'id': 930488995892494336, 'id_str': '93048899..."
3,NaN,None,2019-03-10 16:46:19,"[15, 29]","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,0.0,low,None,1.104786e+18,1.104786e+18,jessicafrota_,1.104785e+18,1.104785e+18,158966715.0,158966715.0,0.0,und,NaN,"{'id': '1ba6cc7d83e56acd', 'url': 'https://api...",NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",@jessicafrota_ @jessicafrota_,2019-03-10 16:46:19.153,0.0,"{'id': 707514822, 'id_str': '707514822', 'name..."
4,NaN,"{'type': 'Point', 'coordinates': [-43.19638889...",2019-03-10 16:45:47,NaN,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,NaN,0.0,0.0,low,"{'type': 'Point', 'coordinates': [-22.91194444...",1.104785e+18,1.104785e+18,None,NaN,NaN,NaN,NaN,0.0,pt,NaN,"{'id': '97bcdfca1a2dca59', 'url': 'https://api...",0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,"<a href=""http://instagram.com"" rel=""nofollow"">...","Emoção é seu 2o nome, Parabéns Paulo Barros. e...",2019-03-10 16:45:47.778,0.0,"{'id': 28799821, 'id_str': '28799821', 'name':..."
5,NaN,None,2019-03-10 16:46:25,NaN,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,0.0,0.0,low,None,1.104786e+18,1.104786e+18,None,NaN,NaN,NaN,NaN,0.0,en,NaN,"{'id': '276336654aa4f87a', 'url': 'https://api...",NaN,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Definitely a chill on the balcony type of day.,2019-03-10 16:46:25.932,0.0,"{'id': 50064363, 'id_str': '50064363', 'name':..."
6,NaN,None,2019-03-10 16:45:10,"[0, 71]","{'hashtags': [], 'urls': [], 'user_mentions': ...","{'media': [{'id': 1104785169339305986, 'id_str...",NaN,0.0,0.0,low,None,1.104785e+18,1.104785e+18,None,NaN,NaN,NaN,NaN,0.0,en,NaN,"{'id': '01fbe706f872cb32', 'url': 'https://api...",0.0,0.0,NaN,NaN,NaN,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Just in case you were feeling optimistic about...,2019-03-10 16:45:10.308,0.0,"{'id': 45929090, 'id_str': '45929090', 'name':..."
7,NaN,None,2019-03-10 16:46:05,NaN,"{'hashtags': [{'text': 'ARSMUN', 'indices': [1...",NaN,NaN,0.0,0.0,low,None,1.104786e+18,1.104786e+18,Non

In [4]:
#Size of dataset
data.shape

(6019, 36)

In [5]:
#Find the null values
data.apply(lambda d: sum(d.isnull()), axis = 0)

contributors                 6019
coordinates                  5439
created_at                     50
display_text_range           3415
entities                       50
extended_entities            5470
extended_tweet               4957
favorite_count                 50
favorited                      50
filter_level                   50
geo                          5439
id                             50
id_str                         50
in_reply_to_screen_name      3726
in_reply_to_status_id        3860
in_reply_to_status_id_str    3860
in_reply_to_user_id          3726
in_reply_to_user_id_str      3726
is_quote_status                50
lang                           50
limit                        5969
place                          55
possibly_sensitive           4621
quote_count                    50
quoted_status                5248
quoted_status_id             5247
quoted_status_id_str         5247
quoted_status_permalink      5248
reply_count                    50
retweet_count 

In [6]:
#Drop columns that may not affect the analysis

if 'limit' in data.columns:
    data.drop(['id', 'created_at', 'contributors', 'display_text_range', 'extended_entities', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'limit', 'quote_count', 'quoted_status_id', 'quoted_status_id_str', 'favorited', 'filter_level', 'is_quote_status', 'quoted_status', 'quoted_status_permalink', 'timestamp_ms', 'truncated'], 1, inplace = True)
else:
    data.drop(['id', 'created_at', 'contributors', 'display_text_range', 'extended_entities', 'id_str', 'in_reply_to_screen_name', 'in_reply_to_status_id', 'in_reply_to_status_id_str', 'in_reply_to_user_id', 'in_reply_to_user_id_str', 'quote_count', 'quoted_status_id', 'quoted_status_id_str', 'favorited', 'filter_level', 'is_quote_status', 'quoted_status', 'quoted_status_permalink', 'timestamp_ms', 'truncated'], 1, inplace = True)
    
data.head(10)

,coordinates,entities,extended_tweet,favorite_count,geo,lang,place,possibly_sensitive,reply_count,retweet_count,retweeted,source,text,user
0,None,"{'hashtags': [], 'urls': [{'url': 'https://t.c...","{'full_text': 'U don't understand my feelings,...",0.0,None,en,"{'id': '7519d5ef0b317d8f', 'url': 'https://api...",NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...","U don't understand my feelings, emotions and m...","{'id': 3772522093, 'id_str': '3772522093', 'na..."
1,None,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",{'full_text': 'Traveling back to Saint Francis...,0.0,None,en,"{'id': '512a8a4a4c4b4be0', 'url': 'https://api...",0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Traveling back to Saint Francis U for the star...,"{'id': 736617558921646080, 'id_str': '73661755..."
2,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,None,en,"{'id': '017a4afa29d71c65', 'url': 'https://api...",NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",don't need your company. I can do things by my...,"{'id': 930488995892494336, 'id_str': '93048899..."
3,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,None,und,"{'id': '1ba6cc7d83e56acd', 'url': 'https://api...",NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",@jessicafrota_ @jessicafrota_,"{'id': 707514822, 'id_str': '707514822', 'name..."
4,"{'type': 'Point', 'coordinates': [-43.19638889...","{'hashtags': [], 'urls': [{'url': 'https://t.c...",NaN,0.0,"{'type': 'Point', 'coordinates': [-22.91194444...",pt,"{'id': '97bcdfca1a2dca59', 'url': 'https://api...",0.0,0.0,0.0,0.0,"<a href=""http://instagram.com"" rel=""nofollow"">...","Emoção é seu 2o nome, Parabéns Paulo Barros. e...","{'id': 28799821, 'id_str': '28799821', 'name':..."
5,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,None,en,"{'id': '276336654aa4f87a', 'url': 'https://api...",NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Definitely a chill on the balcony type of day.,"{'id': 50064363, 'id_str': '50064363', 'name':..."
6,None,"{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,0.0,None,en,"{'id': '01fbe706f872cb32', 'url': 'https://api...",0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Just in case you were feeling optimistic about...,"{'id': 45929090, 'id_str': '45929090', 'name':..."
7,None,"{'hashtags': [{'text': 'ARSMUN', 'indices': [1...",NaN,0.0,None,en,"{'id': '4393349f368f67a1', 'url': 'https://api...",NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dgea was 100% paid to let tht in. Ive seen tha...,"{'id': 4253028928, 'id_str': '4253028928', 'na..."
8,None,"{'hashtags': [{'text': 'Victory', 'indices': [...",{'full_text': '#Victory! Your #TexasColts earn...,0.0,None,en,"{'id': '361610d5aa1f6719', 'url': 'https://api...",0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",#Victory! Your #TexasColts earned a hard-fough...,"{'id': 36268786, 'id_str': '36268786', 'name':..."
9,None,"{'hashtags': [], 'urls': [{'url': 'https://t.c...",{'full_text': 'Dirumah sama di perantauan beda...,0.0,None,in,"{'id': '02a5a8983ca22b6b', 'url': 'https://api...",NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dirumah sama di perantauan beda atuh neng.\nMa...,"{'id': 1675682534, 'id_str': '1675682534', 'na..."


In [7]:
#Find out what is in the entities column
freqs = data.entities.head(5)
for k, v in freqs.items():
    print(f'{k} {v}')

0 {'hashtags': [], 'urls': [{'url': 'https://t.co/s9cBWMDK5u', 'expanded_url': 'https://twitter.com/i/web/status/1104785554368094208', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}], 'user_mentions': [{'screen_name': 'mani', 'name': 'Usman', 'id': 649483, 'id_str': '649483', 'indices': [85, 90]}], 'symbols': []}
1 {'hashtags': [], 'urls': [{'url': 'https://t.co/G8fBX6rjNy', 'expanded_url': 'https://twitter.com/i/web/status/1104785428195016704', 'display_url': 'twitter.com/i/web/status/1…', 'indices': [117, 140]}], 'user_mentions': [{'screen_name': 'RedFlashFB', 'name': 'Saint Francis U FB', 'id': 342323173, 'id_str': '342323173', 'indices': [51, 62]}], 'symbols': []}
2 {'hashtags': [], 'urls': [], 'user_mentions': [], 'symbols': []}
3 {'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'jessicafrota_', 'name': 'Maria', 'id': 158966715, 'id_str': '158966715', 'indices': [0, 14]}, {'screen_name': 'jessicafrota_', 'name': 'Maria', 'id': 158966715, 'id_str

In [8]:
#Find out what is in the extended_tweet column
freqs = data.extended_tweet.head(10)
for k, v in freqs.items():
    print(f'{k} {v}')

0 {'full_text': "U don't understand my feelings, emotions and me,😪😪\nAnd I can't explain on u ...😔😔😔😔😔\n@mani # some one .....\n.............\n...................", 'display_text_range': [0, 141], 'entities': {'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'mani', 'name': 'Usman', 'id': 649483, 'id_str': '649483', 'indices': [85, 90]}], 'symbols': []}}
1 {'full_text': "Traveling back to Saint Francis U for the start of @RedFlashFB spring practice this week.  Can't wait to see this QB to WR connection‼️ https://t.co/kTZlOdh7fZ", 'display_text_range': [0, 135], 'entities': {'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'RedFlashFB', 'name': 'Saint Francis U FB', 'id': 342323173, 'id_str': '342323173', 'indices': [51, 62]}], 'symbols': [], 'media': [{'id': 1104785414479663104, 'id_str': '1104785414479663104', 'indices': [136, 159], 'media_url': 'http://pbs.twimg.com/media/D1T8eycX0AAVYUf.jpg', 'media_url_https': 'https://pbs.twimg.com/media/D1T8eycX0AAVYUf.j

In [9]:
data.drop(['entities', 'extended_tweet'], 1, inplace = True)
data.head(10)

,coordinates,favorite_count,geo,lang,place,possibly_sensitive,reply_count,retweet_count,retweeted,source,text,user
0,None,0.0,None,en,"{'id': '7519d5ef0b317d8f', 'url': 'https://api...",NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...","U don't understand my feelings, emotions and m...","{'id': 3772522093, 'id_str': '3772522093', 'na..."
1,None,0.0,None,en,"{'id': '512a8a4a4c4b4be0', 'url': 'https://api...",0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Traveling back to Saint Francis U for the star...,"{'id': 736617558921646080, 'id_str': '73661755..."
2,None,0.0,None,en,"{'id': '017a4afa29d71c65', 'url': 'https://api...",NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",don't need your company. I can do things by my...,"{'id': 930488995892494336, 'id_str': '93048899..."
3,None,0.0,None,und,"{'id': '1ba6cc7d83e56acd', 'url': 'https://api...",NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",@jessicafrota_ @jessicafrota_,"{'id': 707514822, 'id_str': '707514822', 'name..."
4,"{'type': 'Point', 'coordinates': [-43.19638889...",0.0,"{'type': 'Point', 'coordinates': [-22.91194444...",pt,"{'id': '97bcdfca1a2dca59', 'url': 'https://api...",0.0,0.0,0.0,0.0,"<a href=""http://instagram.com"" rel=""nofollow"">...","Emoção é seu 2o nome, Parabéns Paulo Barros. e...","{'id': 28799821, 'id_str': '28799821', 'name':..."
5,None,0.0,None,en,"{'id': '276336654aa4f87a', 'url': 'https://api...",NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Definitely a chill on the balcony type of day.,"{'id': 50064363, 'id_str': '50064363', 'name':..."
6,None,0.0,None,en,"{'id': '01fbe706f872cb32', 'url': 'https://api...",0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Just in case you were feeling optimistic about...,"{'id': 45929090, 'id_str': '45929090', 'name':..."
7,None,0.0,None,en,"{'id': '4393349f368f67a1', 'url': 'https://api...",NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dgea was 100% paid to let tht in. Ive seen tha...,"{'id': 4253028928, 'id_str': '4253028928', 'na..."
8,None,0.0,None,en,"{'id': '361610d5aa1f6719', 'url': 'https://api...",0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",#Victory! Your #TexasColts earned a hard-fough...,"{'id': 36268786, 'id_str': '36268786', 'name':..."
9,None,0.0,None,in,"{'id': '02a5a8983ca22b6b', 'url': 'https://api...",NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dirumah sama di perantauan beda atuh neng.\nMa...,"{'id': 1675682534, 'id_str': '1675682534', 'na..."


In [10]:
#Drop place column because it indicates where the tweet is associated with but not necessarily originating from
data.drop(['place'], 1, inplace = True)
data.head(10)

,coordinates,favorite_count,geo,lang,possibly_sensitive,reply_count,retweet_count,retweeted,source,text,user
0,None,0.0,None,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...","U don't understand my feelings, emotions and m...","{'id': 3772522093, 'id_str': '3772522093', 'na..."
1,None,0.0,None,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Traveling back to Saint Francis U for the star...,"{'id': 736617558921646080, 'id_str': '73661755..."
2,None,0.0,None,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",don't need your company. I can do things by my...,"{'id': 930488995892494336, 'id_str': '93048899..."
3,None,0.0,None,und,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",@jessicafrota_ @jessicafrota_,"{'id': 707514822, 'id_str': '707514822', 'name..."
4,"{'type': 'Point', 'coordinates': [-43.19638889...",0.0,"{'type': 'Point', 'coordinates': [-22.91194444...",pt,0.0,0.0,0.0,0.0,"<a href=""http://instagram.com"" rel=""nofollow"">...","Emoção é seu 2o nome, Parabéns Paulo Barros. e...","{'id': 28799821, 'id_str': '28799821', 'name':..."
5,None,0.0,None,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Definitely a chill on the balcony type of day.,"{'id': 50064363, 'id_str': '50064363', 'name':..."
6,None,0.0,None,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Just in case you were feeling optimistic about...,"{'id': 45929090, 'id_str': '45929090', 'name':..."
7,None,0.0,None,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dgea was 100% paid to let tht in. Ive seen tha...,"{'id': 4253028928, 'id_str': '4253028928', 'na..."
8,None,0.0,None,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",#Victory! Your #TexasColts earned a hard-fough...,"{'id': 36268786, 'id_str': '36268786', 'name':..."
9,None,0.0,None,in,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dirumah sama di perantauan beda atuh neng.\nMa...,"{'id': 1675682534, 'id_str': '1675682534', 'na..."


In [11]:
#Find out what is in the coordinates column
freqs = data.coordinates.head(10)
for k, v in freqs.items():
    print(f'{k} {v}')

0 None
1 None
2 None
3 None
4 {'type': 'Point', 'coordinates': [-43.19638889, -22.91194444]}
5 None
6 None
7 None
8 None
9 None


In [12]:
#Find out what is in the geo column
freqs = data.geo.head(10)
for k, v in freqs.items():
    print(f'{k} {v}')

0 None
1 None
2 None
3 None
4 {'type': 'Point', 'coordinates': [-22.91194444, -43.19638889]}
5 None
6 None
7 None
8 None
9 None


In [13]:
#Drop geo as it contains what is in the coordinates
data.drop(['geo'], 1, inplace = True)
data.head(10)

,coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,source,text,user
0,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...","U don't understand my feelings, emotions and m...","{'id': 3772522093, 'id_str': '3772522093', 'na..."
1,None,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Traveling back to Saint Francis U for the star...,"{'id': 736617558921646080, 'id_str': '73661755..."
2,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",don't need your company. I can do things by my...,"{'id': 930488995892494336, 'id_str': '93048899..."
3,None,0.0,und,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",@jessicafrota_ @jessicafrota_,"{'id': 707514822, 'id_str': '707514822', 'name..."
4,"{'type': 'Point', 'coordinates': [-43.19638889...",0.0,pt,0.0,0.0,0.0,0.0,"<a href=""http://instagram.com"" rel=""nofollow"">...","Emoção é seu 2o nome, Parabéns Paulo Barros. e...","{'id': 28799821, 'id_str': '28799821', 'name':..."
5,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Definitely a chill on the balcony type of day.,"{'id': 50064363, 'id_str': '50064363', 'name':..."
6,None,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Just in case you were feeling optimistic about...,"{'id': 45929090, 'id_str': '45929090', 'name':..."
7,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dgea was 100% paid to let tht in. Ive seen tha...,"{'id': 4253028928, 'id_str': '4253028928', 'na..."
8,None,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",#Victory! Your #TexasColts earned a hard-fough...,"{'id': 36268786, 'id_str': '36268786', 'name':..."
9,None,0.0,in,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dirumah sama di perantauan beda atuh neng.\nMa...,"{'id': 1675682534, 'id_str': '1675682534', 'na..."


In [14]:
#Create a new coordinates column to hold the coordinatesit
data['geo_coordinates'] = data.coordinates

#Rename coordinates column to geo_tag to indicate whether the tweet has been geo-tagged
data.rename(columns={'coordinates':'geo_tag'}, inplace=True)
data.head(10)

,geo_tag,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,source,text,user,geo_coordinates
0,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...","U don't understand my feelings, emotions and m...","{'id': 3772522093, 'id_str': '3772522093', 'na...",None
1,None,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Traveling back to Saint Francis U for the star...,"{'id': 736617558921646080, 'id_str': '73661755...",None
2,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",don't need your company. I can do things by my...,"{'id': 930488995892494336, 'id_str': '93048899...",None
3,None,0.0,und,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",@jessicafrota_ @jessicafrota_,"{'id': 707514822, 'id_str': '707514822', 'name...",None
4,"{'type': 'Point', 'coordinates': [-43.19638889...",0.0,pt,0.0,0.0,0.0,0.0,"<a href=""http://instagram.com"" rel=""nofollow"">...","Emoção é seu 2o nome, Parabéns Paulo Barros. e...","{'id': 28799821, 'id_str': '28799821', 'name':...","{'type': 'Point', 'coordinates': [-43.19638889..."
5,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Definitely a chill on the balcony type of day.,"{'id': 50064363, 'id_str': '50064363', 'name':...",None
6,None,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Just in case you were feeling optimistic about...,"{'id': 45929090, 'id_str': '45929090', 'name':...",None
7,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dgea was 100% paid to let tht in. Ive seen tha...,"{'id': 4253028928, 'id_str': '4253028928', 'na...",None
8,None,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",#Victory! Your #TexasColts earned a hard-fough...,"{'id': 36268786, 'id_str': '36268786', 'name':...",None
9,None,0.0,in,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dirumah sama di perantauan beda atuh neng.\nMa...,"{'id': 1675682534, 'id_str': '1675682534', 'na...",None


In [15]:
#Re-order columns
if len(data.columns) == 10:
    column_order = [0,9,1,2,3,4,5,6,7,8]
else:
    column_order = [0,10,1,2,3,4,5,6,7,8,9]
data = data[data.columns[column_order]]
data.head(10)

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,source,text,user
0,None,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...","U don't understand my feelings, emotions and m...","{'id': 3772522093, 'id_str': '3772522093', 'na..."
1,None,None,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Traveling back to Saint Francis U for the star...,"{'id': 736617558921646080, 'id_str': '73661755..."
2,None,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",don't need your company. I can do things by my...,"{'id': 930488995892494336, 'id_str': '93048899..."
3,None,None,0.0,und,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",@jessicafrota_ @jessicafrota_,"{'id': 707514822, 'id_str': '707514822', 'name..."
4,"{'type': 'Point', 'coordinates': [-43.19638889...","{'type': 'Point', 'coordinates': [-43.19638889...",0.0,pt,0.0,0.0,0.0,0.0,"<a href=""http://instagram.com"" rel=""nofollow"">...","Emoção é seu 2o nome, Parabéns Paulo Barros. e...","{'id': 28799821, 'id_str': '28799821', 'name':..."
5,None,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Definitely a chill on the balcony type of day.,"{'id': 50064363, 'id_str': '50064363', 'name':..."
6,None,None,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Just in case you were feeling optimistic about...,"{'id': 45929090, 'id_str': '45929090', 'name':..."
7,None,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dgea was 100% paid to let tht in. Ive seen tha...,"{'id': 4253028928, 'id_str': '4253028928', 'na..."
8,None,None,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",#Victory! Your #TexasColts earned a hard-fough...,"{'id': 36268786, 'id_str': '36268786', 'name':..."
9,None,None,0.0,in,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dirumah sama di perantauan beda atuh neng.\nMa...,"{'id': 1675682534, 'id_str': '1675682534', 'na..."


In [16]:
#Encode the geo_tag column
data['geo_tag'].fillna('False', inplace=True)
data.loc[data['geo_tag'] != 'False', 'geo_tag'] = 'True'
data.head(10)

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,source,text,user
0,False,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...","U don't understand my feelings, emotions and m...","{'id': 3772522093, 'id_str': '3772522093', 'na..."
1,False,None,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Traveling back to Saint Francis U for the star...,"{'id': 736617558921646080, 'id_str': '73661755..."
2,False,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",don't need your company. I can do things by my...,"{'id': 930488995892494336, 'id_str': '93048899..."
3,False,None,0.0,und,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",@jessicafrota_ @jessicafrota_,"{'id': 707514822, 'id_str': '707514822', 'name..."
4,True,"{'type': 'Point', 'coordinates': [-43.19638889...",0.0,pt,0.0,0.0,0.0,0.0,"<a href=""http://instagram.com"" rel=""nofollow"">...","Emoção é seu 2o nome, Parabéns Paulo Barros. e...","{'id': 28799821, 'id_str': '28799821', 'name':..."
5,False,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Definitely a chill on the balcony type of day.,"{'id': 50064363, 'id_str': '50064363', 'name':..."
6,False,None,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Just in case you were feeling optimistic about...,"{'id': 45929090, 'id_str': '45929090', 'name':..."
7,False,None,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dgea was 100% paid to let tht in. Ive seen tha...,"{'id': 4253028928, 'id_str': '4253028928', 'na..."
8,False,None,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",#Victory! Your #TexasColts earned a hard-fough...,"{'id': 36268786, 'id_str': '36268786', 'name':..."
9,False,None,0.0,in,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dirumah sama di perantauan beda atuh neng.\nMa...,"{'id': 1675682534, 'id_str': '1675682534', 'na..."


In [17]:
#Obtain the coordinates key-value pair from the previous coordinates column
tags = data['geo_coordinates'].apply(pd.Series)
tags = tags.rename(columns = lambda x : 'tag_' + str(x))
tags

S:\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'str' and 'int', sort order is undefined for incomparable objects
  result = result.union(other)
S:\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:69: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  index = _union_indexes(indexes, sort=sort)
S:\Anaconda3\lib\site-packages\pandas\core\indexes\api.py:107: RuntimeWarning: '<' not supported between instances of 'int' and 'str', sort order is undefined for incomparable objects
  result = result.union(other)


,tag_type,tag_coordinates,tag_0
0,NaN,NaN,NaN
1,NaN,NaN,NaN
2,NaN,NaN,NaN
3,NaN,NaN,NaN
4,Point,"[-43.19638889, -22.91194444]",NaN
5,NaN,NaN,NaN
6,NaN,NaN,NaN
7,NaN,NaN,NaN
8,NaN,NaN,NaN
9,NaN,NaN,NaN


In [18]:
#Assign the new value for coordinates only to the column geo_coordinates created
data['geo_coordinates'] = tags.tag_coordinates
data.head(10)

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,source,text,user
0,False,NaN,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...","U don't understand my feelings, emotions and m...","{'id': 3772522093, 'id_str': '3772522093', 'na..."
1,False,NaN,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Traveling back to Saint Francis U for the star...,"{'id': 736617558921646080, 'id_str': '73661755..."
2,False,NaN,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",don't need your company. I can do things by my...,"{'id': 930488995892494336, 'id_str': '93048899..."
3,False,NaN,0.0,und,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",@jessicafrota_ @jessicafrota_,"{'id': 707514822, 'id_str': '707514822', 'name..."
4,True,"[-43.19638889, -22.91194444]",0.0,pt,0.0,0.0,0.0,0.0,"<a href=""http://instagram.com"" rel=""nofollow"">...","Emoção é seu 2o nome, Parabéns Paulo Barros. e...","{'id': 28799821, 'id_str': '28799821', 'name':..."
5,False,NaN,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",Definitely a chill on the balcony type of day.,"{'id': 50064363, 'id_str': '50064363', 'name':..."
6,False,NaN,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Just in case you were feeling optimistic about...,"{'id': 45929090, 'id_str': '45929090', 'name':..."
7,False,NaN,0.0,en,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dgea was 100% paid to let tht in. Ive seen tha...,"{'id': 4253028928, 'id_str': '4253028928', 'na..."
8,False,NaN,0.0,en,0.0,0.0,0.0,0.0,"<a href=""http://twitter.com/download/iphone"" r...",#Victory! Your #TexasColts earned a hard-fough...,"{'id': 36268786, 'id_str': '36268786', 'name':..."
9,False,NaN,0.0,in,NaN,0.0,0.0,0.0,"<a href=""http://twitter.com/download/android"" ...",Dirumah sama di perantauan beda atuh neng.\nMa...,"{'id': 1675682534, 'id_str': '1675682534', 'na..."


In [19]:
#View the full source field
pd.set_option('display.max_colwidth', -1)
data.source

0       <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
1       <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>  
2       <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
3       <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>  
4       <a href="http://instagram.com" rel="nofollow">Instagram</a>                         
5       <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>  
6       <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
7       <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
8       <a href="http://twitter.com/download/iphone" rel="nofollow">Twitter for iPhone</a>  
9       <a href="http://twitter.com/download/android" rel="nofollow">Twitter for Android</a>
10      <a href="http://twitter.com/download/android" rel="nofollow">T

In [20]:
data.loc[4118]

geo_tag               False                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             

In [21]:
#A hypothesis at this point is that tweeting from an external app adds the geo-tag by default
#hence influences chances of a tweet being geo-tagged

#Therefore obtain meaningful information from the source column

#In the source column, the particular device/app is in between > and < i.e   (......> Device / App <......)
#I will use > and < as delimiters to obtain the device/app name alone

#Split the source column first based on >
first_split = data['source'].str.split(">", n = 1, expand = True)

#Obtain the second part of what was split and since its stored as an array the second is index 1
second_part = first_split[1]
second_part.head(10)

0    Twitter for Android</a>
1    Twitter for iPhone</a> 
2    Twitter for Android</a>
3    Twitter for iPhone</a> 
4    Instagram</a>          
5    Twitter for iPhone</a> 
6    Twitter for Android</a>
7    Twitter for Android</a>
8    Twitter for iPhone</a> 
9    Twitter for Android</a>
Name: 1, dtype: object

In [22]:
#Split the variable holding the data above i.e. second_part

#Split using the < as the delimiter
last = second_part.str.split("<", n = 1, expand = True)

#Obtain the first part of the split which now contains the particular device/app alone and since its stored as an array the first is index 0
specific_names = last[0]
specific_names.head(10)

0    Twitter for Android
1    Twitter for iPhone 
2    Twitter for Android
3    Twitter for iPhone 
4    Instagram          
5    Twitter for iPhone 
6    Twitter for Android
7    Twitter for Android
8    Twitter for iPhone 
9    Twitter for Android
Name: 0, dtype: object

In [23]:
#Assign the particular device/app names as the new source column values and rename it
data.source = specific_names

#Reset column length and print
pd.reset_option('display.max_colwidth')
data.head(10)

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,source,text,user
0,False,NaN,0.0,en,NaN,0.0,0.0,0.0,Twitter for Android,"U don't understand my feelings, emotions and m...","{'id': 3772522093, 'id_str': '3772522093', 'na..."
1,False,NaN,0.0,en,0.0,0.0,0.0,0.0,Twitter for iPhone,Traveling back to Saint Francis U for the star...,"{'id': 736617558921646080, 'id_str': '73661755..."
2,False,NaN,0.0,en,NaN,0.0,0.0,0.0,Twitter for Android,don't need your company. I can do things by my...,"{'id': 930488995892494336, 'id_str': '93048899..."
3,False,NaN,0.0,und,NaN,0.0,0.0,0.0,Twitter for iPhone,@jessicafrota_ @jessicafrota_,"{'id': 707514822, 'id_str': '707514822', 'name..."
4,True,"[-43.19638889, -22.91194444]",0.0,pt,0.0,0.0,0.0,0.0,Instagram,"Emoção é seu 2o nome, Parabéns Paulo Barros. e...","{'id': 28799821, 'id_str': '28799821', 'name':..."
5,False,NaN,0.0,en,NaN,0.0,0.0,0.0,Twitter for iPhone,Definitely a chill on the balcony type of day.,"{'id': 50064363, 'id_str': '50064363', 'name':..."
6,False,NaN,0.0,en,0.0,0.0,0.0,0.0,Twitter for Android,Just in case you were feeling optimistic about...,"{'id': 45929090, 'id_str': '45929090', 'name':..."
7,False,NaN,0.0,en,NaN,0.0,0.0,0.0,Twitter for Android,Dgea was 100% paid to let tht in. Ive seen tha...,"{'id': 4253028928, 'id_str': '4253028928', 'na..."
8,False,NaN,0.0,en,0.0,0.0,0.0,0.0,Twitter for iPhone,#Victory! Your #TexasColts earned a hard-fough...,"{'id': 36268786, 'id_str': '36268786', 'name':..."
9,False,NaN,0.0,in,NaN,0.0,0.0,0.0,Twitter for Android,Dirumah sama di perantauan beda atuh neng.\nMa...,"{'id': 1675682534, 'id_str': '1675682534', 'na..."


In [24]:
#Further cleanup of the source column

#Assign every field with android, iphone or mobile field as mobile to show that it a mobile phone(handheld) 
data['source'] = data.source.str.replace(r'(^.*(Android|iPhone|Mobile).*$)', 'mobile')

data['source'] = data.source.str.replace(r'(^.*Web.*$)', 'web')
data.head(10)

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,source,text,user
0,False,NaN,0.0,en,NaN,0.0,0.0,0.0,mobile,"U don't understand my feelings, emotions and m...","{'id': 3772522093, 'id_str': '3772522093', 'na..."
1,False,NaN,0.0,en,0.0,0.0,0.0,0.0,mobile,Traveling back to Saint Francis U for the star...,"{'id': 736617558921646080, 'id_str': '73661755..."
2,False,NaN,0.0,en,NaN,0.0,0.0,0.0,mobile,don't need your company. I can do things by my...,"{'id': 930488995892494336, 'id_str': '93048899..."
3,False,NaN,0.0,und,NaN,0.0,0.0,0.0,mobile,@jessicafrota_ @jessicafrota_,"{'id': 707514822, 'id_str': '707514822', 'name..."
4,True,"[-43.19638889, -22.91194444]",0.0,pt,0.0,0.0,0.0,0.0,Instagram,"Emoção é seu 2o nome, Parabéns Paulo Barros. e...","{'id': 28799821, 'id_str': '28799821', 'name':..."
5,False,NaN,0.0,en,NaN,0.0,0.0,0.0,mobile,Definitely a chill on the balcony type of day.,"{'id': 50064363, 'id_str': '50064363', 'name':..."
6,False,NaN,0.0,en,0.0,0.0,0.0,0.0,mobile,Just in case you were feeling optimistic about...,"{'id': 45929090, 'id_str': '45929090', 'name':..."
7,False,NaN,0.0,en,NaN,0.0,0.0,0.0,mobile,Dgea was 100% paid to let tht in. Ive seen tha...,"{'id': 4253028928, 'id_str': '4253028928', 'na..."
8,False,NaN,0.0,en,0.0,0.0,0.0,0.0,mobile,#Victory! Your #TexasColts earned a hard-fough...,"{'id': 36268786, 'id_str': '36268786', 'name':..."
9,False,NaN,0.0,in,NaN,0.0,0.0,0.0,mobile,Dirumah sama di perantauan beda atuh neng.\nMa...,"{'id': 1675682534, 'id_str': '1675682534', 'na..."


In [25]:
#Find out the unique elements in source column
data.source.unique()

array(['mobile', 'Instagram', 'web', 'Foursquare', 'Koinchu 5', nan,
       'Twitter for iPad', 'Foursquare Swarm', 'TTN NYC traffic',
       'Untappd', 'CareerArc 2.0', 'OLIO Articles Plugin',
       'Mira Streambox', 'Tweetbot for iΟS', 'pullmeoutspbk',
       'FOL Deutschland', 'eLobbyist', 'ijg', 'Endomondo',
       'City of Portland Fire/EMS feed', 'everyEarthquake', 'srvnotif',
       'Every Finnish Number', 'SafeTweet by TweetMyJOBS', 'FailraBot',
       'Hootsuite Inc.', 'BB RADIO Music App', 'Cities',
       'TTN ORL traffic', 'dlvr.it', 'FL', 'World Cities',
       'Trendsmap Alerting', 'GooGuns Lulz', 'City of Portland 911 feed',
       'every-tract', 'iembot', 'Tweetbot for Mac', 'Adoradores Online',
       'Tandata', 'TTN BOS traffic', '511NY-Tweets', 'Echofon'],
      dtype=object)

In [26]:
data.loc[data['source'] == 33]

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,source,text,user


In [27]:
# #Encode every other source as other
# data.loc[(data['source'] != 'web') & (data['source'] != 'mobile'), 'source'] = 'other'
# data.head(10)

data.source.unique()

array(['mobile', 'Instagram', 'web', 'Foursquare', 'Koinchu 5', nan,
       'Twitter for iPad', 'Foursquare Swarm', 'TTN NYC traffic',
       'Untappd', 'CareerArc 2.0', 'OLIO Articles Plugin',
       'Mira Streambox', 'Tweetbot for iΟS', 'pullmeoutspbk',
       'FOL Deutschland', 'eLobbyist', 'ijg', 'Endomondo',
       'City of Portland Fire/EMS feed', 'everyEarthquake', 'srvnotif',
       'Every Finnish Number', 'SafeTweet by TweetMyJOBS', 'FailraBot',
       'Hootsuite Inc.', 'BB RADIO Music App', 'Cities',
       'TTN ORL traffic', 'dlvr.it', 'FL', 'World Cities',
       'Trendsmap Alerting', 'GooGuns Lulz', 'City of Portland 911 feed',
       'every-tract', 'iembot', 'Tweetbot for Mac', 'Adoradores Online',
       'Tandata', 'TTN BOS traffic', '511NY-Tweets', 'Echofon'],
      dtype=object)

In [28]:
#Find out what is in the user column
freqs = data.user.head(1)
for k, v in freqs.items():
    print(f'{k} {v}')

0 {'id': 3772522093, 'id_str': '3772522093', 'name': 'Er. M.B. Pandey', 'screen_name': 'manibhushanp6', 'location': 'Lucknow, India', 'url': None, 'description': "Works  at CIVIL ENGINEER   \n\nStudies at APJKTU LUCKNOW\n(success is nt matter or luck or its origin, \n  it's only depends upon hardworking nd be honesty )", 'translator_type': 'none', 'protected': False, 'verified': False, 'followers_count': 182, 'friends_count': 129, 'listed_count': 0, 'favourites_count': 3801, 'statuses_count': 1565, 'created_at': 'Sat Oct 03 17:56:34 +0000 2015', 'utc_offset': None, 'time_zone': None, 'geo_enabled': True, 'lang': 'en', 'contributors_enabled': False, 'is_translator': False, 'profile_background_color': 'C0DEED', 'profile_background_image_url': 'http://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_image_url_https': 'https://abs.twimg.com/images/themes/theme1/bg.png', 'profile_background_tile': False, 'profile_link_color': '1DA1F2', 'profile_sidebar_border_color': 'C0DEED'

In [29]:
#Split the user object
tags = data['user'].apply(pd.Series)
tags = tags.rename(columns = lambda x : str(x))
tags

,contributors_enabled,created_at,default_profile,default_profile_image,description,favourites_count,follow_request_sent,followers_count,following,friends_count,geo_enabled,id,id_str,is_translator,lang,listed_count,location,name,notifications,profile_background_color,profile_background_image_url,profile_background_image_url_https,profile_background_tile,profile_banner_url,profile_image_url,profile_image_url_https,profile_link_color,profile_sidebar_border_color,profile_sidebar_fill_color,profile_text_color,profile_use_background_image,protected,screen_name,statuses_count,time_zone,translator_type,url,utc_offset,verified,0
0,False,Sat Oct 03 17:56:34 +0000 2015,True,False,Works at CIVIL ENGINEER \n\nStudies at APJK...,3801.0,NaN,182.0,NaN,129.0,True,3.772522e+09,3772522093,False,en,0.0,"Lucknow, India",Er. M.B. Pandey,NaN,C0DEED,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,False,https://pbs.twimg.com/profile_banners/37725220...,http://pbs.twimg.com/profile_images/1104775447...,https://pbs.twimg.com/profile_images/110477544...,1DA1F2,C0DEED,DDEEF6,333333,True,False,manibhushanp6,1565.0,NaN,none,None,NaN,False,NaN
1,False,Sat May 28 17:58:06 +0000 2016,True,False,"Christian, Husband of Sherry, Proud Dad of Tyl...",782.0,NaN,198.0,NaN,198.0,True,7.366176e+17,736617558921646080,False,en,0.0,Pittsburgh ➡️No.VA ➡️Charlotte,David Shearer,NaN,F5F8FA,,,False,https://pbs.twimg.com/profile_banners/73661755...,http://pbs.twimg.com/profile_images/1073062263...,https://pbs.twimg.com/profile_images/107306226...,1DA1F2,C0DEED,DDEEF6,333333,True,False,David_Shearer10,171.0,NaN,none,http://www.hudl.com/profile/4002264/tyler-shearer,NaN,False,NaN
2,False,Tue Nov 14 17:33:59 +0000 2017,True,False,that was 5:55 p.m.// Your Bhimby Vaklang twooo...,5622.0,NaN,207.0,NaN,342.0,True,9.304890e+17,930488995892494336,False,en,1.0,nowhere,Carl,NaN,F5F8FA,,,False,https://pbs.twimg.com/profile_banners/93048899...,http://pbs.twimg.com/profile_images/1096967284...,https://pbs.twimg.com/profile_images/109696728...,1DA1F2,C0DEED,DDEEF6,333333,True,False,ehmanyuwel,4180.0,NaN,none,None,NaN,False,NaN
3,False,Fri Jul 20 17:35:43 +0000 2012,False,False,se me seguir eh pq quer me dá,32131.0,NaN,18946.0,NaN,14548.0,True,7.075148e+08,707514822,False,pt,75.0,None,mari,NaN,FF6699,http://abs.twimg.com/images/themes/theme1/bg.png,https://abs.twimg.com/images/themes/theme1/bg.png,True,https://pbs.twimg.com/profile_banners/70751482...,http://pbs.twimg.com/profile_images/1104374080...,https://pbs.twimg.com/profile_images/110437408...,9E08F5,C0DEED,DDEEF6,333333,True,False,maripierris,14580.0,NaN,none,None,NaN,False,NaN
4,False,Sat Apr 04 14:18:27 +0000 2009,False,False,Advogada,100.0,NaN,340.0,NaN,1331.0,True,2.879982e+07,28799821,False,pt,8.0,"-22.91913,-43.22563",ANA MASCARENHAS,NaN,B2DFDA,http://abs.twimg.com/images/themes/theme4/bg.gif,https://abs.twimg.com/images/themes/theme4/bg.gif,True,https://pbs.twimg.com/profile_banners/28799821...,http://pbs.twimg.com/profile_images/1002995023...,https://pbs.twimg.com/profile_images/100299502...,F5ABB5,FFFFFF,FFFFFF,333333,True,False,anamascarenhas,6112.0,NaN,none,None,NaN,False,NaN
5,False,Tue Jun 23 18:54:07 +0000 2009,False,False,Same name on Instagram.,1713.0,NaN,1351.0,NaN,1050.0,True,5.006436e+07,50064363,False,en,3.0,"Carencro, Louisiana",Zay,NaN,131516,http://abs.twimg.com/images/themes/theme14/bg.gif,https://abs.twimg.com/images/themes/theme14/bg...,True,NaN,http://pbs.twimg.com/profile_images/1097689470...,https://pbs.twimg.com/profile_images/109768947...,009999,EEEEEE,EFEFEF,333333,True,False,ZayIsKool,47651.0,NaN,none,None,NaN,False,NaN
6,False,Tue Jun 09 20:05:01 +0000 2009,False,False,Experience Designer at @PublicisSapient | Prev...,20146.0,NaN,1285.0,NaN,1019.0,True,4.592909e+07,45929090,False,en,201.0,"Washington, DC",Rachel Grossman,NaN,EDECE9,http://abs.twimg.com/images/themes/theme15/bg.png,https://abs.twimg.com/images/themes/theme15/bg...,False,https://pbs.twi

In [30]:
#Add columns of interest to the dataset from the user object
data['protected'] = tags.protected
data['tweets_liked'] = tags.favourites_count
data['followers'] = tags.followers_count
data['friends'] = tags.friends_count
data['listed_under'] = tags.listed_count
data['geo_enabled'] = tags.geo_enabled
data['user_location'] = tags.location

data.head(10)

,geo_tag,geo_coordinates,favorite_count,lang,possibly_sensitive,reply_count,retweet_count,retweeted,source,text,user,protected,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,False,NaN,0.0,en,NaN,0.0,0.0,0.0,mobile,"U don't understand my feelings, emotions and m...","{'id': 3772522093, 'id_str': '3772522093', 'na...",False,3801.0,182.0,129.0,0.0,True,"Lucknow, India"
1,False,NaN,0.0,en,0.0,0.0,0.0,0.0,mobile,Traveling back to Saint Francis U for the star...,"{'id': 736617558921646080, 'id_str': '73661755...",False,782.0,198.0,198.0,0.0,True,Pittsburgh ➡️No.VA ➡️Charlotte
2,False,NaN,0.0,en,NaN,0.0,0.0,0.0,mobile,don't need your company. I can do things by my...,"{'id': 930488995892494336, 'id_str': '93048899...",False,5622.0,207.0,342.0,1.0,True,nowhere
3,False,NaN,0.0,und,NaN,0.0,0.0,0.0,mobile,@jessicafrota_ @jessicafrota_,"{'id': 707514822, 'id_str': '707514822', 'name...",False,32131.0,18946.0,14548.0,75.0,True,None
4,True,"[-43.19638889, -22.91194444]",0.0,pt,0.0,0.0,0.0,0.0,Instagram,"Emoção é seu 2o nome, Parabéns Paulo Barros. e...","{'id': 28799821, 'id_str': '28799821', 'name':...",False,100.0,340.0,1331.0,8.0,True,"-22.91913,-43.22563"
5,False,NaN,0.0,en,NaN,0.0,0.0,0.0,mobile,Definitely a chill on the balcony type of day.,"{'id': 50064363, 'id_str': '50064363', 'name':...",False,1713.0,1351.0,1050.0,3.0,True,"Carencro, Louisiana"
6,False,NaN,0.0,en,0.0,0.0,0.0,0.0,mobile,Just in case you were feeling optimistic about...,"{'id': 45929090, 'id_str': '45929090', 'name':...",False,20146.0,1285.0,1019.0,201.0,True,"Washington, DC"
7,False,NaN,0.0,en,NaN,0.0,0.0,0.0,mobile,Dgea was 100% paid to let tht in. Ive seen tha...,"{'id': 4253028928, 'id_str': '4253028928', 'na...",False,237.0,189.0,128.0,8.0,True,"London, England"
8,False,NaN,0.0,en,0.0,0.0,0.0,0.0,mobile,#Victory! Your #TexasColts earned a hard-fough...,"{'id': 36268786, 'id_str': '36268786', 'name':...",False,5.0,28.0,27.0,1.0,True,HBU | Husky Stadium | Texas
9,False,NaN,0.0,in,NaN,0.0,0.0,0.0,mobile,Dirumah sama di perantauan beda atuh neng.\nMa...,"{'id': 1675682534, 'id_str': '1675682534', 'na...",False,249.0,252.0,87.0,1.0,True,Medan


In [31]:
#Drop unnecessary columns
if 'retweeted_status' in data:
    data.drop(['text','user','retweeted_status','geo_coordinates','lang'], 1, inplace = True)
else:
    data.drop(['text','user','geo_coordinates','lang'], 1, inplace = True)
data.head(10)

,geo_tag,favorite_count,possibly_sensitive,reply_count,retweet_count,retweeted,source,protected,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,False,0.0,NaN,0.0,0.0,0.0,mobile,False,3801.0,182.0,129.0,0.0,True,"Lucknow, India"
1,False,0.0,0.0,0.0,0.0,0.0,mobile,False,782.0,198.0,198.0,0.0,True,Pittsburgh ➡️No.VA ➡️Charlotte
2,False,0.0,NaN,0.0,0.0,0.0,mobile,False,5622.0,207.0,342.0,1.0,True,nowhere
3,False,0.0,NaN,0.0,0.0,0.0,mobile,False,32131.0,18946.0,14548.0,75.0,True,None
4,True,0.0,0.0,0.0,0.0,0.0,Instagram,False,100.0,340.0,1331.0,8.0,True,"-22.91913,-43.22563"
5,False,0.0,NaN,0.0,0.0,0.0,mobile,False,1713.0,1351.0,1050.0,3.0,True,"Carencro, Louisiana"
6,False,0.0,0.0,0.0,0.0,0.0,mobile,False,20146.0,1285.0,1019.0,201.0,True,"Washington, DC"
7,False,0.0,NaN,0.0,0.0,0.0,mobile,False,237.0,189.0,128.0,8.0,True,"London, England"
8,False,0.0,0.0,0.0,0.0,0.0,mobile,False,5.0,28.0,27.0,1.0,True,HBU | Husky Stadium | Texas
9,False,0.0,NaN,0.0,0.0,0.0,mobile,False,249.0,252.0,87.0,1.0,True,Medan


In [32]:
#Rename columns for easier reference
data.rename(columns={'favorite_count': 'likes', 'reply_count': 'replies', 'retweet_count': 'retweets'}, inplace=True)

In [33]:
#Some useful functions

#Function to encode values
def encode_col(col):
    data[col] = data[col].astype('category')
    data[col] = data[col].cat.codes
    return data.head(10)

#Function to replace null entries with any choice
def impute_nulls(col, null_replacement, data_type):
    data[col].fillna(null_replacement, inplace=True)
    data[col] = data[col].astype(data_type)
    return data.head(10)

In [34]:
#DISADVANTAGE OF LIVE STREAMING TWEETS
#The following columns are dropped as the tweets were streamed live and no data will have been obtained from it

data.drop(['likes','replies', 'retweets', 'retweeted'], 1, inplace = True)
data.head(10)

,geo_tag,possibly_sensitive,source,protected,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,False,NaN,mobile,False,3801.0,182.0,129.0,0.0,True,"Lucknow, India"
1,False,0.0,mobile,False,782.0,198.0,198.0,0.0,True,Pittsburgh ➡️No.VA ➡️Charlotte
2,False,NaN,mobile,False,5622.0,207.0,342.0,1.0,True,nowhere
3,False,NaN,mobile,False,32131.0,18946.0,14548.0,75.0,True,None
4,True,0.0,Instagram,False,100.0,340.0,1331.0,8.0,True,"-22.91913,-43.22563"
5,False,NaN,mobile,False,1713.0,1351.0,1050.0,3.0,True,"Carencro, Louisiana"
6,False,0.0,mobile,False,20146.0,1285.0,1019.0,201.0,True,"Washington, DC"
7,False,NaN,mobile,False,237.0,189.0,128.0,8.0,True,"London, England"
8,False,0.0,mobile,False,5.0,28.0,27.0,1.0,True,HBU | Husky Stadium | Texas
9,False,NaN,mobile,False,249.0,252.0,87.0,1.0,True,Medan


In [35]:
#Function call: Replace null values with 0
impute_nulls('possibly_sensitive', 0, 'int64')

,geo_tag,possibly_sensitive,source,protected,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,False,0,mobile,False,3801.0,182.0,129.0,0.0,True,"Lucknow, India"
1,False,0,mobile,False,782.0,198.0,198.0,0.0,True,Pittsburgh ➡️No.VA ➡️Charlotte
2,False,0,mobile,False,5622.0,207.0,342.0,1.0,True,nowhere
3,False,0,mobile,False,32131.0,18946.0,14548.0,75.0,True,None
4,True,0,Instagram,False,100.0,340.0,1331.0,8.0,True,"-22.91913,-43.22563"
5,False,0,mobile,False,1713.0,1351.0,1050.0,3.0,True,"Carencro, Louisiana"
6,False,0,mobile,False,20146.0,1285.0,1019.0,201.0,True,"Washington, DC"
7,False,0,mobile,False,237.0,189.0,128.0,8.0,True,"London, England"
8,False,0,mobile,False,5.0,28.0,27.0,1.0,True,HBU | Husky Stadium | Texas
9,False,0,mobile,False,249.0,252.0,87.0,1.0,True,Medan


In [36]:
data.possibly_sensitive.unique()

array([0, 1], dtype=int64)

In [37]:
#Function call: Encode source column
encode_col('source')

,geo_tag,possibly_sensitive,source,protected,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,False,0,38,False,3801.0,182.0,129.0,0.0,True,"Lucknow, India"
1,False,0,38,False,782.0,198.0,198.0,0.0,True,Pittsburgh ➡️No.VA ➡️Charlotte
2,False,0,38,False,5622.0,207.0,342.0,1.0,True,nowhere
3,False,0,38,False,32131.0,18946.0,14548.0,75.0,True,None
4,True,0,17,False,100.0,340.0,1331.0,8.0,True,"-22.91913,-43.22563"
5,False,0,38,False,1713.0,1351.0,1050.0,3.0,True,"Carencro, Louisiana"
6,False,0,38,False,20146.0,1285.0,1019.0,201.0,True,"Washington, DC"
7,False,0,38,False,237.0,189.0,128.0,8.0,True,"London, England"
8,False,0,38,False,5.0,28.0,27.0,1.0,True,HBU | Husky Stadium | Texas
9,False,0,38,False,249.0,252.0,87.0,1.0,True,Medan


In [38]:
data.protected.unique()

array([False, nan], dtype=object)

In [39]:
#The following column is dropped because it only contains False or null values
data.drop(['protected'], 1, inplace = True)
data.head(10)

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,False,0,38,3801.0,182.0,129.0,0.0,True,"Lucknow, India"
1,False,0,38,782.0,198.0,198.0,0.0,True,Pittsburgh ➡️No.VA ➡️Charlotte
2,False,0,38,5622.0,207.0,342.0,1.0,True,nowhere
3,False,0,38,32131.0,18946.0,14548.0,75.0,True,None
4,True,0,17,100.0,340.0,1331.0,8.0,True,"-22.91913,-43.22563"
5,False,0,38,1713.0,1351.0,1050.0,3.0,True,"Carencro, Louisiana"
6,False,0,38,20146.0,1285.0,1019.0,201.0,True,"Washington, DC"
7,False,0,38,237.0,189.0,128.0,8.0,True,"London, England"
8,False,0,38,5.0,28.0,27.0,1.0,True,HBU | Husky Stadium | Texas
9,False,0,38,249.0,252.0,87.0,1.0,True,Medan


In [40]:
#Function call: Encode geo_tag column
encode_col('geo_tag')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,38,3801.0,182.0,129.0,0.0,True,"Lucknow, India"
1,0,0,38,782.0,198.0,198.0,0.0,True,Pittsburgh ➡️No.VA ➡️Charlotte
2,0,0,38,5622.0,207.0,342.0,1.0,True,nowhere
3,0,0,38,32131.0,18946.0,14548.0,75.0,True,None
4,1,0,17,100.0,340.0,1331.0,8.0,True,"-22.91913,-43.22563"
5,0,0,38,1713.0,1351.0,1050.0,3.0,True,"Carencro, Louisiana"
6,0,0,38,20146.0,1285.0,1019.0,201.0,True,"Washington, DC"
7,0,0,38,237.0,189.0,128.0,8.0,True,"London, England"
8,0,0,38,5.0,28.0,27.0,1.0,True,HBU | Husky Stadium | Texas
9,0,0,38,249.0,252.0,87.0,1.0,True,Medan


In [41]:
#Function call: Replace null values with 0
impute_nulls('tweets_liked', 0, 'int64')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,38,3801,182.0,129.0,0.0,True,"Lucknow, India"
1,0,0,38,782,198.0,198.0,0.0,True,Pittsburgh ➡️No.VA ➡️Charlotte
2,0,0,38,5622,207.0,342.0,1.0,True,nowhere
3,0,0,38,32131,18946.0,14548.0,75.0,True,None
4,1,0,17,100,340.0,1331.0,8.0,True,"-22.91913,-43.22563"
5,0,0,38,1713,1351.0,1050.0,3.0,True,"Carencro, Louisiana"
6,0,0,38,20146,1285.0,1019.0,201.0,True,"Washington, DC"
7,0,0,38,237,189.0,128.0,8.0,True,"London, England"
8,0,0,38,5,28.0,27.0,1.0,True,HBU | Husky Stadium | Texas
9,0,0,38,249,252.0,87.0,1.0,True,Medan


In [42]:
#Find out arrangement of the tweets_liked values so as to create categories
data.tweets_liked.describe()

count      6019.000000
mean      15913.288088
std       38369.993701
min           0.000000
25%         939.000000
50%        4462.000000
75%       15405.000000
max      532044.000000
Name: tweets_liked, dtype: float64

In [43]:
#Categorize tweets_liked
# data.tweets_liked = pd.cut(data['tweets_liked'], bins=[0, 158965, 317930, 476895, 635860, 794828], include_lowest=True, labels=['very_low', 'low', 'medium', 'high', 'very_high'])
# data.head(10)

In [44]:
#Encode the column
#encode_col('tweets_liked')

In [45]:
impute_nulls('followers', 0, 'int64')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,38,3801,182,129.0,0.0,True,"Lucknow, India"
1,0,0,38,782,198,198.0,0.0,True,Pittsburgh ➡️No.VA ➡️Charlotte
2,0,0,38,5622,207,342.0,1.0,True,nowhere
3,0,0,38,32131,18946,14548.0,75.0,True,None
4,1,0,17,100,340,1331.0,8.0,True,"-22.91913,-43.22563"
5,0,0,38,1713,1351,1050.0,3.0,True,"Carencro, Louisiana"
6,0,0,38,20146,1285,1019.0,201.0,True,"Washington, DC"
7,0,0,38,237,189,128.0,8.0,True,"London, England"
8,0,0,38,5,28,27.0,1.0,True,HBU | Husky Stadium | Texas
9,0,0,38,249,252,87.0,1.0,True,Medan


In [46]:
#Find out arrangement of the followers values so as to create categories
data.followers.describe()

count    6.019000e+03
mean     3.535935e+03
std      4.943615e+04
min      0.000000e+00
25%      1.930000e+02
50%      5.270000e+02
75%      1.347000e+03
max      2.706577e+06
Name: followers, dtype: float64

In [47]:
# #Categorize followers
# data.followers = pd.cut(data['followers'], bins=[0, 1000, 10000, 100000, 1000000, 2900000], include_lowest=True, labels=['very_low', 'low', 'medium', 'high', 'very_high'])
# data.head(10)

In [48]:
# #Encode the column
# encode_col('followers')

In [49]:
impute_nulls('friends', 0, 'int64')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,38,3801,182,129,0.0,True,"Lucknow, India"
1,0,0,38,782,198,198,0.0,True,Pittsburgh ➡️No.VA ➡️Charlotte
2,0,0,38,5622,207,342,1.0,True,nowhere
3,0,0,38,32131,18946,14548,75.0,True,None
4,1,0,17,100,340,1331,8.0,True,"-22.91913,-43.22563"
5,0,0,38,1713,1351,1050,3.0,True,"Carencro, Louisiana"
6,0,0,38,20146,1285,1019,201.0,True,"Washington, DC"
7,0,0,38,237,189,128,8.0,True,"London, England"
8,0,0,38,5,28,27,1.0,True,HBU | Husky Stadium | Texas
9,0,0,38,249,252,87,1.0,True,Medan


In [50]:
#Find out arrangement of the friends values so as to create categories
data.friends.describe()

count      6019.000000
mean       1181.344409
std        4764.849091
min           0.000000
25%         222.000000
50%         481.000000
75%        1016.000000
max      189431.000000
Name: friends, dtype: float64

In [51]:
#Categorize friends
# data.friends = pd.cut(data['friends'], bins=[0, 71487, 142974, 214461, 285948, 357435], include_lowest=True, labels=['very_low', 'low', 'medium', 'high', 'very_high'])
# data.head(10)

In [52]:
#Encode the column
# encode_col('friends')

In [53]:
impute_nulls('listed_under', 0, 'int64')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,38,3801,182,129,0,True,"Lucknow, India"
1,0,0,38,782,198,198,0,True,Pittsburgh ➡️No.VA ➡️Charlotte
2,0,0,38,5622,207,342,1,True,nowhere
3,0,0,38,32131,18946,14548,75,True,None
4,1,0,17,100,340,1331,8,True,"-22.91913,-43.22563"
5,0,0,38,1713,1351,1050,3,True,"Carencro, Louisiana"
6,0,0,38,20146,1285,1019,201,True,"Washington, DC"
7,0,0,38,237,189,128,8,True,"London, England"
8,0,0,38,5,28,27,1,True,HBU | Husky Stadium | Texas
9,0,0,38,249,252,87,1,True,Medan


In [54]:
data.listed_under.describe()

count    6019.000000
mean       30.652766
std       168.672240
min         0.000000
25%         0.000000
50%         3.000000
75%        17.000000
max      6942.000000
Name: listed_under, dtype: float64

In [55]:
#Categorize listed_under
# data.listed_under = pd.cut(data['listed_under'], bins=[0, 3, 14, 32220], include_lowest=True, labels=['low', 'medium', 'high'])
# data.head(10)

In [56]:
# encode_col('listed_under')

In [57]:
encode_col('geo_enabled')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,38,3801,182,129,0,0,"Lucknow, India"
1,0,0,38,782,198,198,0,0,Pittsburgh ➡️No.VA ➡️Charlotte
2,0,0,38,5622,207,342,1,0,nowhere
3,0,0,38,32131,18946,14548,75,0,None
4,1,0,17,100,340,1331,8,0,"-22.91913,-43.22563"
5,0,0,38,1713,1351,1050,3,0,"Carencro, Louisiana"
6,0,0,38,20146,1285,1019,201,0,"Washington, DC"
7,0,0,38,237,189,128,8,0,"London, England"
8,0,0,38,5,28,27,1,0,HBU | Husky Stadium | Texas
9,0,0,38,249,252,87,1,0,Medan


In [58]:
data['user_location'].fillna('False', inplace=True)
data.loc[data['user_location'] != 'False', 'user_location'] = 'True'
data.head(10)

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,38,3801,182,129,0,0,True
1,0,0,38,782,198,198,0,0,True
2,0,0,38,5622,207,342,1,0,True
3,0,0,38,32131,18946,14548,75,0,False
4,1,0,17,100,340,1331,8,0,True
5,0,0,38,1713,1351,1050,3,0,True
6,0,0,38,20146,1285,1019,201,0,True
7,0,0,38,237,189,128,8,0,True
8,0,0,38,5,28,27,1,0,True
9,0,0,38,249,252,87,1,0,True


In [59]:
encode_col('user_location')

,geo_tag,possibly_sensitive,source,tweets_liked,followers,friends,listed_under,geo_enabled,user_location
0,0,0,38,3801,182,129,0,0,1
1,0,0,38,782,198,198,0,0,1
2,0,0,38,5622,207,342,1,0,1
3,0,0,38,32131,18946,14548,75,0,0
4,1,0,17,100,340,1331,8,0,1
5,0,0,38,1713,1351,1050,3,0,1
6,0,0,38,20146,1285,1019,201,0,1
7,0,0,38,237,189,128,8,0,1
8,0,0,38,5,28,27,1,0,1
9,0,0,38,249,252,87,1,0,1


In [60]:
#Bring in the model
X = np.array(data.drop(['geo_tag'], 1).astype(float))
y = np.array(data['geo_tag'])

#Scale the data to remove influence of outliers
scaler = MinMaxScaler()
X = scaler.fit_transform(X)

kmeans = KMeans(n_clusters=2, max_iter=600)
kmeans.fit(X)

correct = 0
for i in range(len(X)):
    predict_me = np.array(X[i].astype(int))
    predict_me = predict_me.reshape(-1, len(predict_me))
    prediction = kmeans.predict(predict_me)
    if prediction[0] == y[i]:
        correct += 1

print((correct/len(X))*100)

75.79332114969264


In [ ]:
#At this point the model's accuracy can be either high or low because the clusters are created without the knowledge 
#of the original classes. Therefore, what was originally a '1' in the geo_tag column can be treated as a '0' by the 
#model as it creates the new classes randomly and does not adhere to our set 1 - geo_tag on and 0 - geo_tag off

In [61]:
#Apply individual columns to the model to see effect
testing_data = data.copy()
y = np.array(testing_data['geo_tag'])
testing_data.drop(['geo_tag'], 1, inplace = True)

for j in range(len(testing_data.columns)):
    X = np.array(testing_data[testing_data.columns[j]].astype(float))

    #Scale the data to remove influence of outliers
    scaler = MinMaxScaler()
    X = scaler.fit_transform(X.reshape(-1, 1))

    kmeans = KMeans(n_clusters=2, max_iter=600)                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                   
    kmeans.fit(X.reshape(-1, 1))

    correct = 0
    for i in range(len(X)):
        predict_me = np.array(X[i].astype(int))
        predict_me = predict_me.reshape(-1, len(predict_me))
        prediction = kmeans.predict(predict_me)
        if prediction[0] == y[i]:
            correct += 1

    print(testing_data.columns[j] , '=>' ,(correct/len(X))*100)

possibly_sensitive => 9.80229273965775
source => 14.387772055158665
tweets_liked => 90.24754942681508
followers => 90.34723375976075
friends => 90.33061970426981
listed_under => 90.34723375976075
geo_enabled => 89.53314504070443
user_location => 75.79332114969264


In [ ]:
#Same case as above applies here as an accuracy of 9% would in essence mean an accuracy of 90/91% because the model 
#might have assigned geo_tag on a '0' instead of what we decided earlier i.e. '1'